# Hyper Parameter Tuning

Using hyper parameter to pick the best version of algorithm

We will use credit card default data from https://www.kaggle.com/datasets/uciml/default-of-credit-card-clients-dataset

References

- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV
- https://www.analyticsvidhya.com/blog/2021/06/tune-hyperparameters-with-gridsearchcv/


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.float_format = '{:,.2f}'.format

## Step-1: Download Data

In [ ]:
import os
import urllib.request

data_url = 'https://raw.githubusercontent.com/elephantscale/datasets/master/credit-card-default/default.csv'
data_location = os.path.basename(data_url)

if not os.path.exists (data_location):
    print("Downloading : ", data_url)
    urllib.request.urlretrieve(data_url, data_location)
print('data_location:', data_location)

In [ ]:
data = pd.read_csv(data_location)
data.sample(10)

## Step-2: EDA

In [ ]:
## Check data skew
data['default'].value_counts()

In [ ]:
data['default'].value_counts(normalize=True)

## Step-3: Clean up 

TODO

## Step-4: Shape data

In [ ]:
label_col = 'default'

feature_columns = data.columns
## TODO : drop 'ID' and 'default' columns
feature_columns = feature_columns.drop (['ID', 'default'])
#print (feature_columns)

In [ ]:
X = data[feature_columns]
y = data[[label_col]]

print (X.shape)
print (y.shape)

## Step-5: Build a Parameter Grid

In [ ]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression

algo = LogisticRegression()

# find out parameters
algo.get_params()

In [ ]:
from sklearn.model_selection import ParameterGrid

# build a param-grid
param_grid =  {
                   # 'penalty' : ['l1', 'l2'],
                    'C' : [0.5, 1.0],
                    'solver' : ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
                     'max_iter' : [50, 100, 200, 300],
              }

Let's try a different algorithm

In [ ]:
# ## Bulid a param grid for RandomForest

# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import ParameterGrid

# algo = RandomForestClassifier()
# algo.get_params()

# # build a param-grid
# param_grid =  {
#                    'n_estimators' : (50, 100, 150),
#                    'max_depth' : (10,15,20,25),
#               }

## Step-6: Grid Search

In [ ]:
%%time 

from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(estimator=algo, 
                             param_grid=param_grid, 
                             cv = 5, 
                             scoring='accuracy',
                             return_train_score=True,
                             n_jobs = -1)

grid_search_results = grid_search.fit (X,np.ravel(y))

## Step-7: Get Grid Search Results

In [ ]:
# get best parameters

best_model = grid_search_results.best_estimator_
print ("Best model : ", best_model)
print ("Best hyper params : ", grid_search_results.best_params_)
print ("Best score : ", grid_search_results.best_score_)

In [ ]:
# internal details

grid_search_results.cv_results_

## Step-8: Evaluate the final model

Now that we have a model from Grid Search, let's see how it does

In [ ]:
from sklearn.linear_model import LogisticRegression

## we can pick the model here or initialize like this
# model = best_model

## or we can initialize using hte params
algo = LogisticRegression(C=0.5, max_iter=300, solver='newton-cg')
model= algo.fit (X, y)


In [ ]:
y_pred = model.predict (X)

In [ ]:
accuracy = model.score(X,y)
print ("Overall accuracy: ", accuracy)

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np

cm_labels = np.unique(y)
cm_array = confusion_matrix(y, y_pred)
#print (cm_array)

cm_df = pd.DataFrame(cm_array, index=cm_labels, columns=cm_labels)
cm_df

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize = (8,5))

# colormaps : cmap="YlGnBu" , cmap="Greens", cmap="Blues",  cmap="Reds"
sns.heatmap(cm_df, annot=True, cmap="Reds", fmt='d').plot()